In [15]:
import pandas as pd
import numpy as np
import os


In [16]:
df = pd.read_csv('CT_2013_2020.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
keep_cols = ['PUB_AGENCY_NAME', 'AGENCY_TYPE_NAME', 'STATE_NAME', 'COUNTY_NAME', 'OFFENSE_NAME', 'PROP_DESC_NAME', 'STOLEN_VALUE'] # set columns we want to keep
drop_cols = [x for x in list(df.columns) if x not in keep_cols] # columns we dont want to keep
drop_cols

['DATA_YEAR',
 'ORI',
 'PUB_AGENCY_UNIT',
 'STATE_ABBR',
 'DIVISION_NAME',
 'REGION_NAME',
 'POPULATION_GROUP_CODE',
 'POPULATION_GROUP_DESC',
 'OFFENSE_CODE',
 'OFFENDER_RACE',
 'OFFENDER_ETHNICITY',
 'OFFENDER_AGE',
 'OFFENDER_SEX',
 'VICTIM_TYPE_CODE',
 'VICTIM_TYPE_NAME',
 'LOCATION_CODE',
 'LOCATION_NAME',
 'WEAPON_CODE',
 'WEAPON_NAME',
 'PROP_DESC_ID',
 'PROP_DESC_CODE',
 'RECOVERED_VALUE',
 'RECOVERED_FLAG',
 'DATE_RECOVERED']

In [18]:
df.drop(inplace=True, columns=drop_cols) # drop columns

In [19]:
df['STOLEN_VALUE'].quantile(q=[0, .95], interpolation='linear') # something useful to see range of values in terms of quantile / quartiles. i.e. 95% of stolen value is 20,000 or less

0.00        0.0
0.95    20000.0
Name: STOLEN_VALUE, dtype: float64

In [20]:
df['value_range'] = None    # set column to change later
df

,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_NAME,COUNTY_NAME,OFFENSE_NAME,PROP_DESC_NAME,STOLEN_VALUE,value_range
0,Franklin,City,Tennessee,WILLIAMSON,Theft From Building,Portable Electronic Communications,3000.0,None
1,Kernersville,City,North Carolina,GUILFORD; FORSYTH,Theft From Building,Purse/ Wallet,50.0,None
2,Whiteville,City,North Carolina,COLUMBUS,Theft From Building,Household Goods,950.0,None
3,Tennessee State University,University or College,Tennessee,NaN,Theft From Building,Portable Electronic Communications,250.0,None
4,Tennessee State University,University or College,Tennessee,NaN,Theft From Building,Portable Electronic Communications,150.0,None
...,...,...,...,...,...,...,...,...
131414,Jacksonville,City,Florida,DUVAL,Theft From Motor Vehicle,Office Equipment,800.0,None
131415,Jacksonville,City,Florida,DUVAL,Theft From Motor Vehicle,Office Equipment,800.0,None
131416,Jacksonville,City,Florida,DUVAL,Theft From Motor Vehicle,Office Equipment,800.0,None
131417,Jacksonville,City,Florida,DUVAL,Theft From Motor Vehicle,Office Equipment,800.0,None


In [21]:
# assign value ranges based on 'STOLEN_VALUE', semi-based on quartile ranges

df.loc[df['STOLEN_VALUE'] < 200, 'value_range'] = '< 200'
df.loc[(df['STOLEN_VALUE'] < 1000) & (df['STOLEN_VALUE'] >= 200), 'value_range'] = '200 - 999'
df.loc[(df['STOLEN_VALUE'] < 2000) & (df['STOLEN_VALUE'] >= 1000), 'value_range'] = '1,000 - 1,999'
df.loc[(df['STOLEN_VALUE'] < 5000) & (df['STOLEN_VALUE'] >= 2000), 'value_range'] = '2,000 - 4,999'
df.loc[(df['STOLEN_VALUE'] < 10000) & (df['STOLEN_VALUE'] >= 5000), 'value_range'] = '5,000 - 9,999'
df.loc[(df['STOLEN_VALUE'] < 20000) & (df['STOLEN_VALUE'] >= 10000), 'value_range'] = '10,000 - 19,999'
df.loc[(df['STOLEN_VALUE'] < 100000) & (df['STOLEN_VALUE'] >= 20000), 'value_range'] = '99,999 - 19,999'
df.loc[(df['STOLEN_VALUE'] > 100000) , 'value_range'] = '> 100,000'

In [22]:
group_cols = ['PUB_AGENCY_NAME', 'AGENCY_TYPE_NAME', 'STATE_NAME', 'OFFENSE_NAME', 'PROP_DESC_NAME', 'value_range', 'COUNTY_NAME']
df = (df
.groupby(group_cols)    # group based on similar column values
.size() # get size of the column == number of similar occurances based on group_cols
.reset_index() # reset index so they are individual columns again
)

In [23]:
df

,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_NAME,OFFENSE_NAME,PROP_DESC_NAME,value_range,COUNTY_NAME,0
0,Abbeville,City,South Carolina,All Other Larceny,Automobile,"5,000 - 9,999",ABBEVILLE,1
1,Abbeville,City,South Carolina,All Other Larceny,Bicycles,200 - 999,ABBEVILLE,1
2,Abbeville,City,South Carolina,All Other Larceny,Clothes/ Furs,< 200,ABBEVILLE,1
3,Abbeville,City,South Carolina,All Other Larceny,Computer Hard/ Software,"2,000 - 4,999",ABBEVILLE,2
4,Abbeville,City,South Carolina,All Other Larceny,Computer Hard/ Software,200 - 999,ABBEVILLE,1
...,...,...,...,...,...,...,...,...
56679,Zavala,County,Texas,Burglary/Breaking & Entering,Other,< 200,ZAVALA,1
56680,Zavala,County,Texas,Motor Vehicle Theft,Automobile,"5,000 - 9,999",ZAVALA,1
56681,Zebulon,City,North Carolina,Theft From Motor Vehicle,Drugs/ Narcotics,"1,000 - 1,999",WAKE,1
56682,Zionsville,City,Indiana,All Other Larceny,Credit/ Debit cards,< 200,BOONE,1


In [24]:
move_pos = df.pop('COUNTY_NAME') # pop this column
df.insert(2, 'COUNTY_NAME', move_pos) # move column to index 2

In [25]:
# same as above
move_pos = df.pop(0)
df.insert(7, 'Number of Occurances', move_pos)

In [26]:
df

,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,COUNTY_NAME,STATE_NAME,OFFENSE_NAME,PROP_DESC_NAME,value_range,Number of Occurances
0,Abbeville,City,ABBEVILLE,South Carolina,All Other Larceny,Automobile,"5,000 - 9,999",1
1,Abbeville,City,ABBEVILLE,South Carolina,All Other Larceny,Bicycles,200 - 999,1
2,Abbeville,City,ABBEVILLE,South Carolina,All Other Larceny,Clothes/ Furs,< 200,1
3,Abbeville,City,ABBEVILLE,South Carolina,All Other Larceny,Computer Hard/ Software,"2,000 - 4,999",2
4,Abbeville,City,ABBEVILLE,South Carolina,All Other Larceny,Computer Hard/ Software,200 - 999,1
...,...,...,...,...,...,...,...,...
56679,Zavala,County,ZAVALA,Texas,Burglary/Breaking & Entering,Other,< 200,1
56680,Zavala,County,ZAVALA,Texas,Motor Vehicle Theft,Automobile,"5,000 - 9,999",1
56681,Zebulon,City,WAKE,North Carolina,Theft From Motor Vehicle,Drugs/ Narcotics,"1,000 - 1,999",1
56682,Zionsville,City,BOONE,Indiana,All Other Larceny,Credit/ Debit cards,< 200,1


In [27]:
try:
    os.mkdir(f'{os.getcwd()}\\cargo_output')
except FileExistsError:
    pass


In [28]:
new_col_names = ['Agency Name', 'Agency Type', 'County Name', 'State Name', 'Offense Name', 'Stolen Item Description', 'Value Range', 'Number of Similar Occurances'] # new column names

df.to_excel(f'{os.getcwd()}\\cargo_output\\cargo_cleaned.xlsx', na_rep = 'NA', sheet_name='Cities', header=new_col_names, index=False) # index=False drops the index column; store cities in separate excel since
